In [1]:
import os
import torch
import string    
import random 
import torchreid
from glob import glob
import os.path as osp
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/home/zaven/anaconda3/envs/face_rec/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
class NewDataset(torchreid.data.datasets.ImageDataset):
    dataset_dir = 'data/custom_camera_data/'

    def __init__(self, **kwargs):
        self.train_dir = self.dataset_dir + "train"  
        self.query_dir = self.dataset_dir + "train"
        self.gallery_dir = self.dataset_dir + "train"
        
        train = self.process_dir(self.train_dir, isQuery=False)
        query = self.process_dir(self.query_dir, isQuery=True)
        gallery = self.process_dir(self.gallery_dir, isQuery=False)

        super(NewDataset, self).__init__(train, query, gallery, **kwargs)
        
    def process_dir(self, dir_path, isQuery):
        img_paths = glob(osp.join(dir_path, '*.png'))
        
        data = []
        for img_path in img_paths:
            img_name = img_path.split('/')[-1]
            name_splitted = img_name.split('_')
            pid = int(name_splitted[0][1:])
            camid = int(name_splitted[1][1:])

            if isQuery:
                camid += 10  # index starts from 0

            data.append((img_path, pid, camid))

        return data

In [3]:
dataset_name = ''.join(random.choices(string.ascii_uppercase + string.digits, k=random.randint(1, 25)))
torchreid.data.register_image_dataset(dataset_name, NewDataset)

datamanager = torchreid.data.ImageDataManager(
    sources=dataset_name, 
    height=250, 
    width=250, 
    batch_size_train=32, 
    batch_size_test=100,
    transforms=["random_flip", "random_crop"],
#     train_sampler='RandomIdentitySampler',  # RandomDatasetSampler RandomIdentitySampler
)

Building train transforms ...
+ resize to 250x250
+ random flip
+ random crop (enlarge to 281x281 and crop 250x250)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 250x250
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded NewDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    15 |    12774 |         1
  query    |    15 |    12774 |         1
  gallery  |    15 |    12774 |         1
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded NewDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    15 |    12774 |         1
  query    |    15 |    12774 |         1
  gal

In [7]:
model = torchreid.models.build_model(
    name="osnet_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="triplet",
    pretrained=True
).to(device).train()

weight_path = f"models/retrain_osnet_x1_0/model/lfw_model.pth.tar-90"
torchreid.utils.load_pretrained_weights(model, weight_path) 

Successfully loaded imagenet pretrained weights from "/home/zaven/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Successfully loaded pretrained weights from "models/retrain_osnet_x1_0/model/lfw_model.pth.tar-90"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [9]:
optim_name = 'adam'  # adam sgd radam
lr = 0.003  # 0.0005 0.0003 0.0001  # DO NOT SET lr MORE THAN 0.01, IT'S CRUSHING FOR FACE, START FROM lr = 0.003 

optimizer = torchreid.optim.build_optimizer(
    model,
    optim=optim_name,
    lr=lr, 
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step", 
    stepsize=20,
)

engine = torchreid.engine.ImageTripletEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    margin=0.3,  # by default 0.3
    weight_t=50,  # weight for triplet loss
    weight_x=1, # weight for softmax loss
)

engine.run(
    save_dir="models/retrain_osnet_x1_0",
    max_epoch=100, 
    eval_freq=10, 
    print_freq=99,
    test_only=False
)

=> Start training
epoch: [1/100][99/399]	time 0.237 (0.229)	data 0.000 (0.009)	eta 2:31:35	loss_t 0.0169 (0.0360)	loss_x 0.7146 (0.8928)	acc 96.8750 (98.9268)	lr 0.003000
epoch: [1/100][198/399]	time 0.240 (0.223)	data 0.001 (0.005)	eta 2:27:26	loss_t 0.0000 (0.0222)	loss_x 0.6190 (0.7747)	acc 100.0000 (99.3845)	lr 0.003000
epoch: [1/100][297/399]	time 0.241 (0.222)	data 0.001 (0.003)	eta 2:26:37	loss_t 0.0000 (0.0156)	loss_x 0.6116 (0.7242)	acc 100.0000 (99.5476)	lr 0.003000
epoch: [1/100][396/399]	time 0.224 (0.221)	data 0.000 (0.003)	eta 2:25:48	loss_t 0.0689 (0.0128)	loss_x 0.6759 (0.6975)	acc 100.0000 (99.6528)	lr 0.003000
epoch: [2/100][99/399]	time 0.221 (0.228)	data 0.000 (0.009)	eta 2:29:43	loss_t 0.0024 (0.0060)	loss_x 0.5968 (0.6194)	acc 100.0000 (99.8737)	lr 0.003000
epoch: [2/100][198/399]	time 0.207 (0.223)	data 0.001 (0.005)	eta 2:26:14	loss_t 0.0000 (0.0038)	loss_x 0.5959 (0.6122)	acc 100.0000 (99.8895)	lr 0.003000
epoch: [2/100][297/399]	time 0.231 (0.222)	data 0.002 (

KeyboardInterrupt: 